In [1]:
import pandas as pd
import numpy 
import matplotlib.pyplot as plt 
from sqlalchemy import create_engine
import os
from math import ceil

In [2]:
os.environ['DJANGO_USER'] = 'awsadmin'
os.environ['DJANGO_PASS'] = 'HBn2a5BSfhRw6yg'
os.environ['DJANGO_HOST'] = 'catalejo.cto4eoy56d57.us-east-1.rds.amazonaws.com'
os.environ['DJANGO_PORT'] = '5432'
os.environ['DJANGO_DB']   = 'django'

engine = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(os.environ['DJANGO_USER'],
                                                                 os.environ['DJANGO_PASS'],
                                                                 os.environ['DJANGO_HOST'],
                                                                 os.environ['DJANGO_PORT'],
                                                                 os.environ['DJANGO_DB']),
                       connect_args={"options": "-c timezone=America/Mexico_City"})

In [3]:
#Queries
units_info = '''select distinct(listing_nickname), 
                list.bathrooms as bathrooms, 
                list.bedrooms as bedrooms
                from api_reservation as res
                join api_listing as list on res.listingid = list.ID
                order by listing_nickname '''

median_nights = ''' select listing_nickname, 
                percentile_disc(0.5) within group (order by nightscount) as nights from api_reservation
                where status= 'confirmed'
                group by listing_nickname'''

In [6]:
amenities = pd.read_csv('./Amenities.csv')

days = 10
nights = pd.read_sql(median_nights, engine)
nights['checkouts'] = days / nights['nights']

units = pd.read_sql(units_info, engine)

units = units.merge(nights.copy(), on = 'listing_nickname').dropna()

units.loc[units['checkouts'] > 7, 'checkouts'] = 7
units.loc[units['checkouts'] < 2, 'checkouts'] = 2
units.head()

,listing_nickname,bathrooms,bedrooms,nights,checkouts
0,AMS 185 - 302,2,2,3,3.333333
1,AMS 297 - 101,2,2,3,3.333333
2,ASOLA 72 - 501,2,2,4,2.500000
3,BEF 175 - 101,1,1,2,5.000000
4,BEF 175 - 102,1,1,3,3.333333


In [7]:
def get_inventory(unit, amenities, is_building = False):
    '''
    unit: This function takes a dataframe with:
    listing_nickname | bathrooms | bedrooms | nights | checkouts
    ------------------------------------------------------------
    
    It may be a unit, or a building with all the 
    values summed up.
    
    amenities: A dataframe of products with:
    Product | Per | Quantity
    ------------------------
    
    is_building: Set to True if it's a building
    
    EXAMPLES:
    
    Unit:
    unit = units[unit['listing_nickname'] == 'ASOLA 72 - 501']
    get_inventory(unit, amenities)
    
    Building:
    building = units[units['listing_nickname'].str.startswith('BEF')].sum()
    get_inventory(building, amenities, True)
    '''
    if isinstance(unit, pd.DataFrame):
        unit = unit.iloc[0] # Series type is required
    
    print('Unit',unit['listing_nickname'])
    print('Days', days)
    print('Nights', unit['nights'])
    print('Checkouts', unit['checkouts'])
    print('Bedrooms',unit['bedrooms'])
    print('Bathrooms',unit['bathrooms'])
    

    amenities.loc[amenities['Per'] == 'Unit', 'Sugg_Inv'] = unit['checkouts'] * amenities['Quantity']
    amenities.loc[amenities['Per'] == 'Bathroom', 'Sugg_Inv'] = unit['checkouts'] * amenities['Quantity'] \
                                                                * unit['bathrooms']
    amenities.loc[amenities['Per'] == 'Bedroom', 'Sugg_Inv'] = unit['checkouts'] * amenities['Quantity'] \
                                                               * unit['bedrooms']
        
        
    amenities['Sugg_Inv'] = amenities['Sugg_Inv'].apply(ceil)
    
    if is_building:
        amenities.to_csv('Inventory Suggested - ' + unit['listing_nickname'][:4] + '.csv', index = False)
    else:
        amenities.to_csv('Inventory Suggested - ' + unit['listing_nickname'] + '.csv', index = False )

In [8]:
unit = units[units['listing_nickname'] == 'ASOLA 72 - 501']
get_inventory(unit, amenities)

Unit ASOLA 72 - 501
Days 10
Nights 4
Checkouts 2.5
Bedrooms 2
Bathrooms 2


In [9]:
building = units[units['listing_nickname'].str.startswith('BEF')].sum()

get_inventory(building, amenities, is_building= True)

Unit BEF 175 - 101BEF 175 - 102BEF 175 - 103BEF 175 - 201BEF 175 - 202BEF 175 - 203BEF 175 - 301BEF 175 - 302BEF 175 - 303BEF 175 - 401BEF 175 - 402BEF 175 - 403
Days 10
Nights 37
Checkouts 40.833333333333336
Bedrooms 12
Bathrooms 12


In [10]:
amenities.sort_values(by = 'Sugg_Inv', ascending= False)

,Product,Per,Quantity,Sugg_Inv
34,Algodones,Bathroom,1.00,490
32,Kleenex,Bathroom,1.00,490
31,Papel Baño,Bathroom,1.00,490
30,Crema Ajolote,Bedroom,1.00,490
29,Kit ajolote,Bedroom,1.00,490
27,Ear Plugs,Bedroom,1.00,490
23,Pasta de dientes,Bedroom,1.00,490
22,Cepillo Dientes,Bedroom,1.00,490
21,Antifaz,Bedroom,1.00,490
33,Bolsas para basura,Unit,5.00,205
